In [14]:
import netket as nk
import netket.experimental as nkx
import netket.nn as nknn

from math import pi

from netket.experimental.operator.fermion import destroy as c
from netket.experimental.operator.fermion import create as cdag
from netket.experimental.operator.fermion import number as nc

from vmc_torch.fermion_utils import generate_random_fpeps, product_bra_state, get_amp
import quimb.tensor as qtn
from autoray import do
import symmray as sr
import pickle
import os
os.environ['NUMBA_NUM_THREADS'] = '20'

# Define the lattice shape
L = 4  # Side of the square
Lx = int(L)
Ly = int(L//2)
# graph = nk.graph.Square(L)
graph = nk.graph.Grid([Lx,Ly], pbc=False)
N = graph.n_nodes


# Define the fermion filling and the Hilbert space
N_f = int(Lx*Ly/2)
hi = nkx.hilbert.SpinOrbitalFermions(N, s=None, n_fermions=N_f)


# Define the Hubbard Hamiltonian
t = 1.0
V = 4.0
H = 0.0
for (i, j) in graph.edges(): # Definition of the Hubbard Hamiltonian
    # print(i,j)
    H -= t * (cdag(hi,i) * c(hi,j) + cdag(hi,j) * c(hi,i))
    H += V * nc(hi,i) * nc(hi,j)


# Exact diagonalization of the Hamiltonian for benchmark
sp_h = H.to_sparse() # Convert the Hamiltonian to a sparse matrix
from scipy.sparse.linalg import eigsh
eig_vals, eig_vecs = eigsh(sp_h, k=2, which="SA")
E_gs = eig_vals[0]
print("Exact ground state energy per site:", E_gs/N, 'Total energy:', E_gs)

Exact ground state energy per site: -0.2033383116428346 Total energy: -1.626706493142677


In [15]:
# SU in quimb
D = 4
seed = 2
symmetry = 'U1'
peps, parity_config = generate_random_fpeps(Lx, Ly, D, seed, symmetry, Nf=N_f)

edges = qtn.edges_2d_square(Lx, Ly)
site_info = sr.utils.parse_edges_to_site_info(
    edges,
    D,
    phys_dim=2,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
)

t = 1.0
V = 4.0
mu = 0.0

terms = {
    (sitea, siteb): sr.fermi_hubbard_spinless_local_array(
        t=t, V=V, mu=mu,
        symmetry=symmetry,
        coordinations=(
            site_info[sitea]['coordination'],
            site_info[siteb]['coordination'],
        ),
    ).fuse((0, 1), (2, 3))
    for (sitea, siteb) in peps.gen_bond_coos()
}
ham = qtn.LocalHam2D(Lx, Ly, terms)
su = qtn.SimpleUpdateGen(peps, ham, compute_energy_per_site=True,D=D, compute_energy_opts={"max_distance":2}, gate_opts={'cutoff':1e-12})

# cluster energies may not be accuracte yet
su.evolve(50, tau=0.3)
# su.evolve(50, tau=0.1)
# su.evolve(100, tau=0.03)
# su.evolve(100, tau=0.01)
# su.evolve(100, tau=0.003)

peps = su.get_state()
peps.equalize_norms_(value=1)


n=50, tau=0.3000, energy~-0.121510: 100%|##########| 50/50 [00:01<00:00, 29.40it/s]


fPEPS(tensors=8, indices=18, Lx=4, Ly=2, max_bond=4)

In [16]:
# Netket Hamiltonian energy
import numpy as np
all_configs = hi.all_states()
psi_vec = np.array([peps.get_amp(config).contract() for config in all_configs])
psi_vec /= np.linalg.norm(psi_vec)
hamiltonian = H.to_dense()
print((psi_vec.conj().T @ (hamiltonian @ psi_vec))/N)

-0.11196752267224776


In [17]:
# Quimb energy
peps.compute_local_expectation(
    ham.terms, normalized=True, max_bond=64,
)/N

np.float64(-0.11196752267224798)

In [18]:
hi.n_orbitals

8